In [171]:
import sys
import numpy as np
import open3d as o3d
import cv2 as cv
import glob
import importlib
import pathlib
from scipy.spatial.transform import Rotation as R
import os

sys.path.append("../../")
from data_processor import IntrinsicCalib, mp4ToPng
from CT_data_registration import o3d_ICP

In [18]:
importlib.reload(IntrinsicCalib)
importlib.reload(mp4ToPng)

<module 'data_processor.mp4ToPng' from '/home/fj/Projects/ARPA-H/Scripts/20241108_TRE/../../data_processor/mp4ToPng.py'>

In [135]:
# Define the dimensions of the checkerboard (number of inner corners per row and column)
CHECKERBOARD_SHAPE = (10, 7)  # Adjust according to your checkerboard dimensions
CHECKERBOARD_GRID_SIZE = 20 # mm

videoBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241011_phantom_mono/Recordings/run_4")
videoPath = videoBasePath / "Oct11_BPH_Recording5.avi"
imageBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241011_phantom_mono/Recordings/run_5/checkerboard/cam0")
imagePath = imageBasePath / "data"
timeStampsPath = videoBasePath / "Oct11_BPH_Recording5.csv"
timeStamps = np.genfromtxt(timeStampsPath, delimiter=",", dtype=np.double)[:, 1]


bphVideoBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241011_phantom_mono/Recordings/run_5")
bphVideoPath = bphVideoBasePath / "Oct11_BPH_Recording6.avi"
bphImageBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241011_phantom_mono/Recordings/run_5/BPH/cam0")
bphImagePath = bphImageBasePath / "data"
bphTimeStampsPath = bphVideoBasePath / "Oct11_BPH_Recording6.csv"
bphTimeStamps = np.genfromtxt(bphTimeStampsPath, delimiter=",", dtype=np.double)[:, 1]

VSLAM_RECON_PATH = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241011_phantom_mono/Recordings/run_5/BPH/slam_reconstruction")


In [172]:
# Define the dimensions of the checkerboard (number of inner corners per row and column)
# CHECKERBOARD = (10, 7)  # Adjust according to your checkerboard dimensions
CHECKERBOARD_SHAPE = (9, 6)  # Adjust according to your checkerboard dimensions
CHECKERBOARD_GRID_SIZE = 20 # mm

videoBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241025_phantom_mono_tre/White/run_cb")
videoPath = videoBasePath / "Oct25_BPH_White_cb.avi"
imageBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241025_phantom_mono_tre/White/run_cb/checkerboard/cam0")
imagePath = imageBasePath / "data"
timeStampsPath = videoBasePath / "Oct25_BPH_White_cb.csv"
timeStamps = np.genfromtxt(timeStampsPath, delimiter=",", dtype=np.double)[:, 1]

bphVideoBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241025_phantom_mono_tre/White/run_2")
bphVideoPath = bphVideoBasePath / "Oct25_BPH_White_2.avi"
bphImageBasePath = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241025_phantom_mono_tre/White/run_2/BPH/cam0")
bphImagePath = bphImageBasePath / "data"
bphTimeStampsPath = bphVideoBasePath / "Oct25_BPH_White_2.csv"
bphTimeStamps = np.genfromtxt(bphTimeStampsPath, delimiter=",", dtype=np.double)[:, 1]

VSLAM_RECON_PATH = pathlib.Path("/media/fj/Data/Projects/ARPA-H/data/20241025_phantom_mono_tre/White/run_2/BPH/slam_reconstruction")


# Preprocessing and Calibration

## Convert video to png frames

In [ ]:
mp4ToPng.saBveFramesAsPng(bphVideoPath, bphImagePath, newSize=(640, 360), timeStamps=bphTimeStamps)
mp4ToPng.saveFramesAsPng(videoPath, imagePath, newSize=(640, 360), timeStamps=timeStamps)

## Calibration

In [174]:
images = glob.glob(str(imagePath / "*.png"))
print(f"# of Images: {len(images)}")
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
ret, camera_matrix, dist_coeffs, rvecs, tvecs = IntrinsicCalib.detectCheckerboardAndIntrinsicCalib(images, CHECKERBOARD_SHAPE=CHECKERBOARD_SHAPE, CHECKERBOARD_GRID_SIZE=20, savePath=imageBasePath / "intrinsics", visualize=True, imageRange=[0, len(images), 7])

# of Images: 1381
1380/1381
Error:
 1.8348587704031785
Camera Matrix:
 [[235.7407498    0.         322.29086211]
 [  0.         235.99207087 210.66387128]
 [  0.           0.           1.        ]]

Distortion Coefficients:
 [[-0.38956039  0.26414248 -0.00994592  0.00697214 -0.20356178]]


In [10]:
str(bphImagePath)

'/media/fj/Data/Projects/ARPA-H/data/20241011_phantom_mono/Recordings/run_5/checkerboard/cam0/data'

# Run DROID-SLAM

For 20241011 (1st) target error run:\
`python demo.py --imagedir=/media/fj/Data/Projects/ARPA-H/data/20241011_phantom_mono/Recordings/run_5/BPH/cam0/data/ --calib=calib/mono_endo_20241011_run4.txt --t0=0 --reconstruction_path /media/fj/Data/Projects/ARPA-H/data/20241011_phantom_mono/Recordings/run_5/BPH/slam_reconstruction`

For 20241025 (2nd) target error run:\
`python demo.py --imagedir=/media/fj/Data/Projects/ARPA-H/data/20241025_phantom_mono_tre/White/run_2/BPH/cam0/data --calib=calib/mono_endo_20241025_white.txt --t0=60 --reconstruction_path /media/fj/Data/Projects/ARPA-H/data/20241025_phantom_mono_tre/White/run_2/BPH/slam_reconstruction`

# Reconstruction Analysis

In [170]:
# vslamCam  = np.load(VSLAM_RECON_PATH / "camera.ply", allow_pickle=True)
vslamPoses = np.load(VSLAM_RECON_PATH / "poses.npy", allow_pickle=True)
vslamFrameStamps = np.load(VSLAM_RECON_PATH / "tstamps.npy", allow_pickle=True)

vslamTimeStamps = bphTimeStamps[vslamFrameStamps.astype(int)]


## ICP regitsration between VSLAM and CT

## Obtain NID track data corressponding to VSLAM Reconstruction

In [166]:
# Load NDI Timestamps and poses
videoFiles = [f for f in os.listdir(bphVideoBasePath) if f.endswith(".avi")]
# ndiFileAlgined = "ndi_1728660745.233532_Oct11_BPH_recording_6_aligned.csv"
ndiFileAlgined = "1729869737.0550358_Oct25_BPH_White_2_aligned.csv"
videoTimeFiles = [f for f in os.listdir(bphVideoBasePath) if (f.split(".")[0]+".avi") in videoFiles and f.endswith(".csv")]
videoTimeFile = videoTimeFiles[0]

ndiTimeStamps = np.genfromtxt(bphVideoBasePath/ndiFileAlgined, delimiter=",")
ndiTimeStamps = ndiTimeStamps[:-1, :]

# Visualizing the ndi tracked cam Poses
ndiCamPositions = np.array(ndiTimeStamps[:, 1:4])
ndiPhantomPositions =np.array(ndiTimeStamps[:, 8:11])
ndiPhantomPositions = ndiPhantomPositions[~ np.any(np.isnan(ndiPhantomPositions), axis=-1), :]


ndiCamPtCloud = o3d.geometry.PointCloud()
ndiCamPtCloud.points = o3d.utility.Vector3dVector(ndiCamPositions)
ndiCamPtCloud.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in ndiCamPositions])


ndiPhantomCloud = o3d.geometry.PointCloud()
ndiPhantomCloud.points = o3d.utility.Vector3dVector(ndiPhantomPositions)
ndiPhantomCloud.colors = o3d.utility.Vector3dVector([[0, 1, 0] for _ in ndiPhantomPositions])


ndiFileAlignedRun1 = "/media/fj/Data/Projects/ARPA-H/data/20241025_phantom_mono_tre/White/run_1/1729869514.5427468_Oct25_BPH_White_1_aligned.csv"
ndiTimeStampsRun1 = np.genfromtxt(ndiFileAlignedRun1, delimiter=",")
ndiTimeStampsRun1 = ndiTimeStampsRun1[:-1, :]
ndiPhantomPositionsRun1 = np.array(ndiTimeStampsRun1[:, 1:4])
ndiPhantomPositionsRun1 = ndiPhantomPositionsRun1[~ np.any(np.isnan(ndiPhantomPositionsRun1), axis=-1), :]

ndiPhantomCloudRun1 = o3d.geometry.PointCloud()
ndiPhantomCloudRun1.points = o3d.utility.Vector3dVector(ndiPhantomPositionsRun1)
ndiPhantomCloudRun1.colors = o3d.utility.Vector3dVector([[0, 0, 1] for _ in ndiTimeStampsRun1])


o3d.visualization.draw_geometries([ndiCamPtCloud, ndiPhantomCloud, ndiPhantomCloudRun1])

# # ndiFiles = [f for f in os.listdir(directory) if f.startswith("ndi") and f.endswith(".csv") and not f.endswith("original.csv") and not f.endswith("aligned.csv")]
# videoFiles = [f for f in os.listdir(bphVideoBasePath) if f.endswith(".avi")]
# ndiFiles = [f for f in os.listdir(bphVideoBasePath) if (f.split(".")[0]+".avi") not in videoFiles and f.endswith(".csv") and not f.endswith("original.csv") and f.endswith("aligned.csv")]
# videoTimeFiles = [f for f in os.listdir(bphVideoBasePath) if (f.split(".")[0]+".avi") in videoFiles and f.endswith(".csv")]


# for ndiFileN in videoTimeFiles:
#     curNDITimeStamps = np.genfromtxt(bphVideoBasePath/ndiFileN, delimiter=",")
#     curNDITimeStamps = curNDITimeStamps[:-1, :]
#     print(f"Start Time: {curNDITimeStamps[0, 0]}\nEnd Time: {curNDITimeStamps[-1, 0]}")
#     print(not (
#         (curNDITimeStamps[-1, 0] <= vslamTimeStamps[0]) or
#         (curNDITimeStamps[0, 0] >= vslamTimeStamps[-1])))

In [167]:
run2PhantomCentroid = np.mean(ndiPhantomPositions, axis=0)
run1PhantomCentroid = np.mean(ndiPhantomPositionsRun1, axis=0)

In [168]:
run2PhantomCentroid

array([  249.7399442 ,   -65.15833678, -1169.05104578])

In [169]:
run1PhantomCentroid

array([  152.24021423,   150.32580052, -1096.81415717])

In [132]:
# Visualizing the vslam Poses
vslamPositions = np.array(vslamPoses[:, :3])

ptCloud = o3d.geometry.PointCloud()
ptCloud.points = o3d.utility.Vector3dVector(vslamPositions)
ptCloud.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in vslamPositions])

o3d.visualization.draw_geometries([ptCloud])

In [133]:
# Obtain ndi poses corresponding to the vslam poses
print(vslamTimeStamps.astype(int))

ndiTrackAtVSLAMKeyFrames = ndiTimeStamps[vslamFrameStamps.astype(int), :]

[1728660759964 1728660760095 1728660760562 1728660760829 1728660761029
 1728660761096 1728660761164 1728660761266 1728660761466 1728660761563
 1728660761697 1728660761797 1728660761866 1728660761930 1728660761997
 1728660762164 1728660762266 1728660762364 1728660762431 1728660762831
 1728660767802 1728660777779 1728660777918 1728660779281 1728660779414
 1728660779520 1728660779617 1728660779683 1728660779750 1728660779817
 1728660779983 1728660780119 1728660780217 1728660780351 1728660780719
 1728660787925 1728660796231 1728660797335 1728660797599 1728660799940]
